<a href="https://colab.research.google.com/github/Disha-Sikka/Maize_Disease_Classification/blob/main/Maize_Disease_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle competitions download -c ssd-3-l-maiize-disease-classification

 72% 116M/161M [00:00<00:00, 1.21GB/s]
100% 161M/161M [00:00<00:00, 996MB/s] 


In [ ]:
import zipfile
zip_ref= zipfile.ZipFile('/content/ssd-3-l-maiize-disease-classification.zip')
zip_ref.extractall('/content')
zip_ref.close()